In [136]:
%load_ext autoreload
%autoreload 2

import cPickle, os, sys
import random
sys.path.insert(1, os.path.join(sys.path[0], '..'))

from ddlite import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Importing documents

In [137]:
#Thomas data folder 
# DATA_FOLDER='/Users/thomaspalomares/Desktop/Stanford/RA/multisentences/data/'
#Tanya data folder
# DATA_FOLDER='/Users/tanyaberardini/Desktop/DeepDive/data/'
DATA_FOLDER='arabidopsis_data/'

In [138]:
import json

In [139]:
# #To be modified to conserve correct doc_ids (in particular, write in different txt files)
# nb_lines=0
# nb_error_parsing=0
# with open(DATA_FOLDER+'pmc/json/output_plant.json', 'rb') as json_file:
#     with open(DATA_FOLDER+'pmc/text/output_plant.text', 'wb') as text_write:
#         docs={}
#         for line in json_file:
#             nb_lines+=1
#             doc_id_read=str(json.loads(line)['doc-id'].encode('utf-8').strip())
# #             try:
#             #Because this doc is too big
#             if doc_id_read=='10.1093.jxb.eru198':
#                 if random.random() <0.5:
#                     doc_id_read += '_1'
#                 else:
#                     doc_id_read += '_2'
#             if doc_id_read in docs:
#                 docs[doc_id_read] += unicode(str(json.loads(line)['content'].encode('utf-8').strip()) + '.\n', errors='ignore')
#             else:
#                 docs[doc_id_read] = unicode(str(json.loads(line)['content'].encode('utf-8').strip()) + '.\n', errors='ignore')
# #                 text_write.write(unicode(str(json.loads(line)['content'].encode('utf-8').strip()) + '.\n', errors='ignore'))
# #             except:
# #                 nb_error_parsing +=1
# #                 print "error parsing"
# print nb_lines
# print nb_error_parsing

In [140]:
# test = SentenceParser()
# test.parse(docs)
# # dp = DocParser(DATA_FOLDER + 'pmc/text/', TextParser())

In [141]:
pkl_f=DATA_FOLDER+'pmc/pickle/pkl_sentences'
sents=[]
count_loop=0
try:
    with open(pkl_f, 'rb') as f:
        sents = cPickle.load(f)
except:
    print"parsing data"
    for doc_id_loop in docs:
        count_loop+=1
        print count_loop
        for j in test.parse(docs[doc_id_loop], doc_id=doc_id_loop):
            sents.append(j)
    with open(pkl_f, 'w+') as f:
        cPickle.dump(sents, f)

In [142]:
print sents[1]

Sentence(words=[u'It', u'Takes', u'Two', u'to', u'Tango', u':', u'A', u'New', u'Partner', u'in', u'Amylose', u'Synthesis', u'.'], lemmas=[u'it', u'take', u'two', u'to', u'Tango', u':', u'A', u'New', u'Partner', u'in', u'Amylose', u'synthesis', u'.'], poses=[u'PRP', u'VBZ', u'CD', u'TO', u'NNP', u':', u'NNP', u'NNP', u'NNP', u'IN', u'NNP', u'NN', u'.'], dep_parents=[2, 0, 2, 5, 2, 5, 9, 9, 5, 12, 12, 9, 2], dep_labels=[u'nsubj', u'ROOT', u'dobj', u'case', u'nmod', u'punct', u'compound', u'compound', u'dep', u'case', u'compound', u'nmod', u'punct'], sent_id=1, doc_id='PBIOLOGY-D-14-02490', text=u'It Takes Two to Tango: A New Partner in Amylose Synthesis.', token_idxs=[153, 156, 162, 166, 169, 174, 176, 178, 182, 190, 193, 201, 210])


In [143]:
#Think about how to add doc_ids. 
#In particular, look at the small code in parseDocSentences()
# %time sents = dp.parseDocSentences()
# print sents[0]

# Candidate mention Extraction

## Gene extraction

In [144]:
#What is the exact schema of genes ?
genes=[]
for row_genes in [line.rstrip().split('\t') for line in open(DATA_FOLDER + '/dicts/list_genes.txt')]:
    for gene in row_genes:
        genes.append(gene)
        
#Removing blacklist_words
blacklist_words = [line.rstrip().split('\t')[1].lower() for line in open(DATA_FOLDER + '/dicts/blacklist_words.txt')][1:]
genes_filtered = [x.lower() for x in genes if x.lower() not in blacklist_words]

gene_dm = DictionaryMatch(label='GeneName', dictionary=genes_filtered, ignore_case=False)

The dictionary match should provide fairly high recall, but we may still miss some candidates. We know that gene names are named nouns and are often all uppercase. Let's use DDLite's compositional matcher operations to handle this. First, we'll write a matcher to find all nouns using the parts-of-speech tags. Then, we'll use a filter to find uppercase sequences. Finally, we'll use a filter to make sure each match has at least 3 characters. We pass noun_rm to up_rm, and up_rm to the final filter to compose them with each other.

In [145]:
noun_regex = RegexNgramMatch(label='Nouns', regex_pattern=r'[A-Z]?NN[A-Z]?', ignore_case=True, match_attrib='poses')
up_regex = RegexFilterAll(noun_regex, label='Upper', regex_pattern=r'[A-Z]+([0-9]+)?([A-Z]+)?([0-9]+)?$', ignore_case=False, match_attrib='words')
multi_regex = RegexFilterAll(up_regex, label='Multi', regex_pattern=r'[a-z0-9]{3,}', ignore_case=True)

Since we want matches both from the dictionary and the uppercase-noun-phrase-matcher we just built, we'll use the union object to create a matcher for both.

In [146]:
#The previous regexes seem to catch many patterns not wanted (like 'and' or others), let's ignore it for now and we can add it when we'll work on recall
# GM = Union(gene_dm, multi_regex)
GM=gene_dm

## Pheno extraction

In [ ]:
# Schema is: HPO_ID | NAME | TYPE (exact, lemma)
phenos_deepdive = [line.rstrip().split('\t')[1] for line in open(DATA_FOLDER + '/dicts/pheno_terms.tsv')]
phenos_arabidopsis = [line.rstrip() for line in open(DATA_FOLDER + '/dicts/list_phenotypes_arabidopsis_filtered.txt')]
phenos_worm_variants = [line.rstrip() for line in open(DATA_FOLDER + '/dicts/worm_variants_unix.txt')]
phenos_all_eq_dict = [pheno.strip() for line in open(DATA_FOLDER + 'dicts/phenotypes_all_eq_dict.txt') for pheno in line.rstrip().split(';')]
phenos_manual = [line.rstrip() for line in open(DATA_FOLDER + '/dicts/phenotypes_manual.txt')]

phenos_tot = phenos_deepdive + phenos_arabidopsis + phenos_worm_variants + phenos_all_eq_dict + phenos_manual

#Removing the phenotypes that are in the blacklist.
blacklist_words = [line.rstrip().split('\t')[1].lower() for line in open(DATA_FOLDER + '/dicts/blacklist_words.txt')][1:]
phenos_tot_filtered = [x.lower() for x in phenos_tot if x.lower() not in blacklist_words]

PM = DictionaryMatch(label='PhenoName', dictionary=phenos_tot_filtered)

# Relation Extraction

For each sentence, we extract a candidate relation for each pair (gene, phenotype) appearing in the sentence.

In [ ]:
R = Relations(sents, GM, PM)

In [ ]:
R[0].render()

In [ ]:
# test_sent=9
# for i in range(len(R[test_sent].dep_parents)):
#     print i, R[test_sent].words[i], R[test_sent].dep_parents[i], R[test_sent].poses[i]

In [ ]:
# R.dump_candidates('pickle/relations.pkl')

# ddlite Model

We create a ddlite model from the previous candidates, on which we will extract features, define labeling functions and learn a model. 

In [ ]:
DDL = DDLiteModel(R)
print "Extracted {} features for each of {} mentions".format(DDL.num_feats(), DDL.num_candidates())

# Building a gold ground truth

We label here a certain amount of sentences that will allow us to evaluate our model.

In [ ]:
#Importing previous mentions labeled
tags_file = DATA_FOLDER + 'tags/tags_fccb81915ae086e3'
first_line=True
tags_list=[]
with open(tags_file, 'rb') as f:
    for line in f:
        if first_line:
            header=line.rstrip().split('\t')
            first_line=False
        else:
            res={}
            for i, item in enumerate(line.rstrip().split('\t')):
                res[header[i]]=item
            res['sent_id']=int(res['sent_id'])
            res['e1_idxs'] = map(int, res['e1_idxs'][1:-1].split(','))
            res['e2_idxs'] = map(int, res['e2_idxs'][1:-1].split(','))
            tags_list.append(res)

In [ ]:
print len(tags_list)
print tags_list[0]

In [ ]:
#Finding the index in DDL of our labeled data
def func_DDL_index_with_tag(DDL_var, tag_var):
#     count=0
    for i in range(DDL.num_candidates()):
        if DDL.C[i].doc_id==tag_var['doc_id']:
            if DDL.C[i].sent_id==tag_var['sent_id']:
                if DDL.C[i].e1_idxs==tag_var['e1_idxs']:
                    if DDL.C[i].e2_idxs==tag_var['e2_idxs']:
#                         count+=1
                        return i
    return -1
#     if count >1:
#         print 'WEIRD: more than one element found'

In [ ]:
#Creating the corresponding gold file
gold = np.zeros((DDL.num_candidates()))
for i in range(len(tags_list)):
    idx = func_DDL_index_with_tag(DDL, tags_list[i])
    if idx > -1:
        if tags_list[i]['is_correct']=='false':
            gold[np.array([idx])] = np.array([-1])
        if tags_list[i]['is_correct']=='true':
            gold[np.array([idx])] = np.array([1])
            
            
DDL.set_gold_labels(gold)
DDL.set_holdout(p=0.5)

In [ ]:
#Yellow: gene. Blue: phenotype
# DDL.open_mindtagger(num_sample=200, width='100%', height=1200)

In [ ]:
# DDL.add_mindtagger_tags()
# DDL.get_ground_truth()

# Writing labeling functions

We'll use data programming to learn a logistic regression model which will predict the probability of a candidate entity being a true gene mention. Since our training data is not manually labeled, we'll generate many (potentially noisy) labels as a surrogate for precise, manual labels. Feature extraction and model learning are very simple in ddlite. Writing labeling functions is where the real artistry comes in. One of ddlite's goals is to enable rapid prototyping, debugging, and experimenting with labeling functions. These can be used either to create a simple standalone app, or to plug into DeepDive. Labeling functions, or LFs, are functions that take an Candidate object. They must return 1 (for a positive label), 0 (for abstaining), or -1 (for a negative example). For now, we'll write a few simple LFs to get started:

In [ ]:
#Here, idx_mention refers to 0 or 1, whether we consider the gene or phenotype
def post_window(r, idx_mention, key, n=3):
    s = list(r.all_idxs[idx_mention])
    b = len(r.lemmas) - np.max(s)
    s.extend([np.max(s) + i for i in range(1, min(b,n+1))])
    return key in [r.lemmas[i] for i in s]
def pre_window (r, idx_mention, key, n=3):
    s = list(r.all_idxs[idx_mention])
    b = np.min(s)
    s.extend([b - i for i in range(1, min(b,n+1))])
    return key in [r.lemmas[i] for i in s]
def stopper(r, idx_mention, stop):
    return stop in [r.lemmas[i] for i in r.all_idxs[idx_mention]]

#If the word mutation is before gene:
def LF_mutation_gene_t(r):
    return 1 if 'mutation' in [r.lemmas[r.dep_parents[i]-1] for i in r.all_idxs[0]] else 0
#If the word mutation is before pheno:
def LF_mutation_pheno_t(r):
    return 1 if 'mutation' in [r.lemmas[r.dep_parents[i]-1] for i in r.all_idxs[1]] else 0
#If phenotype just before gene, certainly wrong
def LF_pheno_before_gene_f(r):
    res=False
    for i in r.all_idxs[1]:
        if pre_window(r, 0, r.words[i], n=1):
            res=True
    if res:
        return -1
    else:
        return 0
#If phenotype is only a verb, likely to be false
def LF_pheno_only_verb_f(r):
    if len(r.all_idxs[1])==1 and r.poses[r.all_idxs[1][0]][0:2] == 'VB':
        return -1
    else:
        return 0
#if phenotype only one word, likely to be false (except lethal)
def LF_pheno_only_one_word_f(r):
    if len(r.all_idxs[1])==1 and r.lemmas[r.all_idxs[1][0]] != 'lethal':
        return -1
    else:
        return 0
#if gene and phenotype have words in common, very likely to be false
def LF_gene_pheno_words_in_common_f(r):
    if len([val for val in r.all_idxs[0] if val in r.all_idxs[1]]) >0:
        return -1
    else:
        return 0
#if gfp at the end of a gene, likely to be false
def LF_gene_ends_in_gfp_f(r):
    if len(r.all_idxs[0])==1 and len(r.mention1()[0]) >2 and r.mention1()[0][-3:].lower() == 'gfp':
        return -1
    else:
        return 0
#If double mutant around the gene name, more likely true:
def LF_double_mutant_gene_t(r):
    if (post_window(r, 0, 'double', 3) and post_window(r, 0, 'mutant', 3)) or (pre_window(r, 0, 'double', 3) and pre_window(r, 0, 'mutant', 3)):
        return 1
    else:
        return 0
#'In WT' is generally not a phenotype
def LF_in_WT_pheno_f(r):
    if len(r.all_idxs[1])==1 and r.words[r.all_idxs[1][0]] == 'WT' and pre_window(r,1, 'in', 1):
        return -1
    else:
        return 0


In [ ]:
LFs= [LF_mutation_gene_t, LF_mutation_pheno_t, LF_pheno_before_gene_f, LF_pheno_only_verb_f, LF_pheno_only_one_word_f, LF_gene_pheno_words_in_common_f, LF_gene_ends_in_gfp_f, LF_double_mutant_gene_t, LF_in_WT_pheno_f]
DDL.apply_lfs(LFs, clear=True)

### More analysis on the labeling functions

In [ ]:
DDL.print_lf_stats()

In [ ]:
DDL.plot_lf_stats()

In [ ]:
DDL.top_conflict_lfs(n=3)

In [ ]:
DDL.lowest_coverage_lfs(n=15)

In [ ]:
DDL.lowest_empirical_accuracy_lfs(n=10)

In [ ]:
#Let's have a look at the labeled elements:
DDL.get_labeled_ground_truth(gt='resolve', subset=None)

In [ ]:
print DDL.lf_matrix

In [ ]:
DDL.C[16]

# Learning

In [ ]:
matplotlib.rcParams['figure.figsize'] = (12,4)
%time DDL.learn_weights(sample=False, maxIter=500, alpha=0, bias=True, verbose=True, log=True)

In [ ]:
%matplotlib inline
DDL.plot_calibration()

# Iterating with labeling functions

In [ ]:
DDL.show_log()

In [ ]:
DDL.open_mindtagger(width='100%', height=1200)

# Attempt with multi sentences

Here we concatenante 2 sentences in a same document, then export all the relations from this new dataset and keep only the candidates that have a "." between them.

First basic version when some end of sentences "." are replaced by ";" to easily have candidates covered in multisentences.

In [ ]:
# #To be modified to conserve correct doc_ids (in particular, write in different txt files)
# nb_lines=0
# nb_error_parsing=0
# with open(DATA_FOLDER+'pmc/json/output_plant.json', 'rb') as json_file:
#     with open(DATA_FOLDER+'pmc/text/output_plant.text', 'wb') as text_write:
#         docs_multisentences={}
#         for line in json_file:
#             nb_lines+=1
#             doc_id_read=str(json.loads(line)['doc-id'].encode('utf-8').strip())
# #             try:
#             #Because this doc is too big
#             if doc_id_read=='10.1093.jxb.eru198':
#                 if random.random() <0.5:
#                     doc_id_read += '_1'
#                 else:
#                     doc_id_read += '_2'
#             if doc_id_read in docs_multisentences:
#                 if random.random() < 0.5:
#                     docs_multisentences[doc_id_read] += unicode(str(json.loads(line)['content'].encode('utf-8').strip()) + '.\n', errors='ignore')
#                 else:
#                     docs_multisentences[doc_id_read] += unicode(str(json.loads(line)['content'].encode('utf-8').strip()) + ';\n', errors='ignore')
#             else:            
#                 if random.random() < 0.5:
#                     docs_multisentences[doc_id_read] = unicode(str(json.loads(line)['content'].encode('utf-8').strip()) + '.\n', errors='ignore')
#                 else:
#                     docs_multisentences[doc_id_read] = unicode(str(json.loads(line)['content'].encode('utf-8').strip()) + ';\n', errors='ignore') 

                    
# print nb_lines
# print nb_error_parsing

In [ ]:
# pkl_f=DATA_FOLDER+'pmc/pickle/pkl_sentences_multi'
# sents_multi=[]
# count_loop=0
# try:
#     with open(pkl_f, 'rb') as f:
#         sents_multi = cPickle.load(f)
# except:
#     print"parsing data"
#     for doc_id_loop in docs:
#         count_loop+=1
#         print count_loop
#         for j in test.parse(docs_multisentences[doc_id_loop], doc_id=doc_id_loop):
#             sents_multi.append(j)
#     with open(pkl_f, 'w+') as f:
#         cPickle.dump(sents_multi, f)

In [ ]:
# print sents_multi[0]

In [ ]:
# R_multi = Relations(sents_multi, GM, PM)

In [ ]:
# DDL_multi = DDLiteModel(R_multi)
# print "Extracted {} features for each of {} mentions".format(DDL_multi.num_feats(), DDL_multi.num_candidates())

In [ ]:
# DDL_multi.open_mindtagger(num_sample=200, width='100%', height=1200)